In [3]:
import json
import numpy as np

In [4]:
import torch
print(torch.__version__)


2.9.0+cu130


In [5]:
def prepare_TStroop(rec, features, TStroop_way = 1):
    if TStroop_way == 1:
        seq_all = []  # 4 подтеста
        for key in ["mono", "trueColor", "color", "trueText"]:
            seq = []
            for row in rec["test_results"][key]:
                values = []
                for f in features:
                    values.append(row[f])  # добавляем в values данные по признакам
                seq.append(values)  # добавляем в seq строку
            seq_all.append(np.array(seq, dtype=float))  # каждый подтест отдельный массив
        return np.array(seq_all), rec["age"]  # массив из 4 таблиц, размерность 4


    if TStroop_way == 2:
        seq =[]
        for key in ["mono", "trueColor", "color", "trueText"]: #всё в одной таблице
            for row in rec["test_results"][key]:
                values = []
                for f in features:    
                    values.append(row[f])    #добавляем в values данные по нужным признакам (строка)
                seq.append(values)       #добавляем в seq строку
    return np.array(seq, dtype=float), rec["age"]

def prepare_test(rec, features):
    seq = []
    for row in rec["test_results"]: #список словарей
        values = []
        for f in features:
            values.append(row[f])    #добавляем в values данные по нужным признакам (строка)
        seq.append(values)       #добавляем в seq строку
    return np.array(seq, dtype=float), rec["age"]

def prepare_data(data, test_name: str, features: list[str], TStroop_way = 1):  #TStroop_way = 1 если разделяем mono и trueColor, 2 если нет

    X_list = []
    y_list = []
    for rec in data:
        if rec["test_name"] == "TStroop" and test_name == "TStroop":
            X_seq, y_val = prepare_TStroop(rec, features, TStroop_way)
            X_list.append(X_seq)  # добавляем либо 3д либо 4д массив
            y_list.append(y_val)  

        elif rec["test_name"] == test_name:
                X_seq, y_val = prepare_test(rec, features)
                X_list.append(X_seq) #добавляем таблицу по одной попытке теста
                y_list.append(y_val) 

    if test_name == "TStroop" and TStroop_way == 1:
        # Обработка для TStroop с разделением на подтесты
        num_subtests = 4  # фиксированное количество подтестов
        max_len = max(seq.shape[1] for seq in X_list)  # максимальная длина подтеста
        num_features = X_list[0].shape[2]
        batch_size = len(X_list)
        
        X = np.zeros((batch_size, num_subtests, max_len, num_features), dtype=float)#  Padding (выравнивание по длине) 
        for i, seq in enumerate(X_list):
            X[i, :, :seq.shape[1], :] = seq#заполняем тензор данными, там где их нет, оставляем 0
    else:
        # Обработка для остальных случаев
        max_len = max(seq.shape[0] for seq in X_list)
        num_features = X_list[0].shape[-1]  # берем последнюю размерность
        batch_size = len(X_list)
        
        X = np.zeros((batch_size, max_len, num_features), dtype=float)
        for i, seq in enumerate(X_list):
            X[i, :seq.shape[0], :] = seq

    y = np.array(y_list, dtype=float)
    return X, y


(batch_size, num_subtests, max_len, num_features) или (batch_size, max_len, num_features)

In [6]:
json_path = "calculator_9_2025-03-19_filter.json"
test_name = ["T1back", "TStroop", "T258", "T274", "T278"]
features_T1back = [
    "Stimul", "Goal", "Color", "Duration",
    "Show", "Hide", "Down", "Up",
    "SMR", "MR", "ERR_1", "ERR_2", "ERR_3"
]
features_TStroop = ["True", "Error", "SMR", "MR", "Time"]
features_T258 = ["Stimul", "H", "H+", "dH+", "H-", "dH-", "t+", "t-", "ERR", "ERR_LIM"]
features_T274 = ["Stimul", "Goal", "Duration", "Interval", "Show", "Hide", "Down", "Up", "SMR", "MR", "ERR_1", "ERR_2", "ERR_3"]
features_T278 = features_T274

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

x, y = prepare_data(data, test_name[1], features_TStroop, 1)    
print(f" Размерность X: {x.shape}, размерность Y: {y.shape}")

 Размерность X: (5040, 4, 29, 5), размерность Y: (5040,)


In [7]:
np.save("X.npy", x) # сохранение в файл
np.save("y.npy", y)



In [8]:
t1back_data = [row for row in data if row["test_name"] == "T1back"] #список словарей

print(f"Найдено {len(t1back_data)} записей для T1back")
t1back_data[1]

Найдено 5040 записей для T1back


{'test_results': [{'#': 0,
   'Stimul': 2,
   'Goal': 0,
   'Color': 5,
   'ColorHTML': '#804040',
   'Place': 1,
   'Size': 1,
   'Duration': 300,
   'Interval': 2000,
   'Show': 2003,
   'Hide': 2316,
   'Down': None,
   'Up': None,
   'SMR': None,
   'MR': None,
   'ERR_1': 0,
   'ERR_2': 0,
   'ERR_3': 0},
  {'#': 1,
   'Stimul': 5,
   'Goal': 0,
   'Color': 2,
   'ColorHTML': '#00ff00',
   'Place': 1,
   'Size': 1,
   'Duration': 300,
   'Interval': 2000,
   'Show': 4016,
   'Hide': 4318,
   'Down': None,
   'Up': None,
   'SMR': None,
   'MR': None,
   'ERR_1': 0,
   'ERR_2': 0,
   'ERR_3': 0},
  {'#': 2,
   'Stimul': 5,
   'Goal': 0,
   'Color': 3,
   'ColorHTML': '#ffff00',
   'Place': 1,
   'Size': 1,
   'Duration': 300,
   'Interval': 2000,
   'Show': 6018,
   'Hide': 6330,
   'Down': None,
   'Up': None,
   'SMR': None,
   'MR': None,
   'ERR_1': 0,
   'ERR_2': 0,
   'ERR_3': 0},
  {'#': 3,
   'Stimul': 2,
   'Goal': 1,
   'Color': 3,
   'ColorHTML': '#ffff00',
   'Place': 1

In [9]:
test_name = set()
tstroop_data = [row for row in data if row["test_name"] == "TStroop"]
for row in tstroop_data:
    for podtest in row["test_results"].keys():
        test_name.add(podtest)

test_name

{'color', 'mono', 'trueColor', 'trueText'}

In [10]:
d = [row for row in data if row["test_name"] == "TStroop"] #список словарей

print(f"Найдено {len(d)} записей для TStroop")
d[1]

Найдено 5040 записей для TStroop


{'test_results': {'mono': [{'#': 1,
    'id': 1,
    'True': 3,
    'Answer': 3,
    'Error': 0,
    'SMR': 4982,
    'MR': 108,
    'Time': 4982},
   {'#': 2,
    'id': 2,
    'True': 5,
    'Answer': 5,
    'Error': 0,
    'SMR': 1095,
    'MR': 118,
    'Time': 1095},
   {'#': 3,
    'id': 3,
    'True': 3,
    'Answer': 3,
    'Error': 0,
    'SMR': 1368,
    'MR': 110,
    'Time': 1368},
   {'#': 4,
    'id': 4,
    'True': 4,
    'Answer': 4,
    'Error': 0,
    'SMR': 840,
    'MR': 96,
    'Time': 840},
   {'#': 5,
    'id': 5,
    'True': 2,
    'Answer': 2,
    'Error': 0,
    'SMR': 1056,
    'MR': 110,
    'Time': 1056},
   {'#': 6,
    'id': 6,
    'True': 4,
    'Answer': 4,
    'Error': 0,
    'SMR': 1016,
    'MR': 102,
    'Time': 1016},
   {'#': 7,
    'id': 7,
    'True': 1,
    'Answer': 1,
    'Error': 0,
    'SMR': 1248,
    'MR': 96,
    'Time': 1248},
   {'#': 8,
    'id': 8,
    'True': 3,
    'Answer': 3,
    'Error': 0,
    'SMR': 984,
    'MR': 94,
    'Time

In [11]:
for row in data:
    if row['id_test_attempt'] > 1:
        print(row['test_name'], row['id_person'], row['id_test_attempt'])

T274 26523 2
T278 26523 2
TStroop 26523 2
T258 26523 2
T1back 26523 2
TStroop 50911 2
T1back 50911 2
T258 50911 2
T274 50911 2
T278 50911 2
T1back 51091 2
T278 51091 2
T258 51091 2
TStroop 51091 2
T274 51091 2
T278 51091 3
T1back 51091 3
T274 51091 3
TStroop 51091 3
T258 51091 3
TStroop 53999 2
T258 53999 2
T1back 53999 2
T274 53999 2
T278 53999 2
T258 54997 2
TStroop 54997 2
T278 54997 2
T1back 54997 2
T274 54997 2
T274 55865 2
T1back 55865 2
T278 55865 2
TStroop 55865 2
T258 55865 2
T258 55865 3
TStroop 55865 3
T278 55865 3
T1back 55865 3
T274 55865 3
T274 55865 4
T1back 55865 4
T258 55865 4
TStroop 55865 4
T278 55865 4
TStroop 55865 5
T278 55865 5
T1back 55865 5
T274 55865 5
T258 55865 5
T1back 57826 2
T258 57826 2
T274 57826 2
T278 57826 2
TStroop 57826 2
TStroop 62292 2
T278 62292 2
T274 62292 2
T258 62292 2
T1back 62292 2
T1back 62774 2
TStroop 62774 2
T274 62774 2
T278 62774 2
T258 62774 2
T1back 64212 2
T258 64212 2
T274 64212 2
TStroop 64212 2
T278 64212 2
T258 64212 3
TStroop

In [12]:
# Чтение .npy файлов
X_T1back = np.load('X_T1back.npy')
X_T258 = np.load('X_T258.npy')
X_T274 = np.load('X_T274.npy')
X_T278 = np.load('X_T278.npy')
X_TStroop = np.load('X_TStroop.npy')
y_aligned = np.load('y_aligned.npy')

# Вывод информации о данных
print( X_TStroop[1])
print("y_aligned values:", y_aligned[:10])  # Первые 10 значений

[[3.000e+00 0.000e+00 4.982e+03 1.080e+02 4.982e+03]
 [5.000e+00 0.000e+00 1.095e+03 1.180e+02 1.095e+03]
 [3.000e+00 0.000e+00 1.368e+03 1.100e+02 1.368e+03]
 [4.000e+00 0.000e+00 8.400e+02 9.600e+01 8.400e+02]
 [2.000e+00 0.000e+00 1.056e+03 1.100e+02 1.056e+03]
 [4.000e+00 0.000e+00 1.016e+03 1.020e+02 1.016e+03]
 [1.000e+00 0.000e+00 1.248e+03 9.600e+01 1.248e+03]
 [3.000e+00 0.000e+00 9.840e+02 9.400e+01 9.840e+02]
 [2.000e+00 0.000e+00 1.328e+03 7.800e+01 1.328e+03]
 [3.000e+00 0.000e+00 8.880e+02 1.020e+02 8.880e+02]
 [4.000e+00 0.000e+00 9.760e+02 9.500e+01 9.760e+02]
 [2.000e+00 0.000e+00 1.208e+03 1.110e+02 1.208e+03]
 [1.000e+00 0.000e+00 1.928e+03 1.010e+02 1.928e+03]
 [6.000e+00 0.000e+00 2.896e+03 1.580e+02 2.896e+03]
 [4.000e+00 0.000e+00 1.215e+03 1.330e+02 1.215e+03]
 [5.000e+00 0.000e+00 1.184e+03 9.400e+01 1.184e+03]
 [3.000e+00 0.000e+00 1.032e+03 1.340e+02 1.032e+03]
 [4.000e+00 0.000e+00 1.087e+03 1.270e+02 1.087e+03]
 [6.000e+00 0.000e+00 1.152e+03 1.180e+02 1.15

In [13]:
from collections import defaultdict

attempts_counter = defaultdict(int)
for row in data:
    attempts_counter[(row['id_person'], row['id_test_attempt'])] += 1

print("Количество уникальных attempt_key:", len(attempts_counter))
print("Пример распределения по попыткам:", sorted(attempts_counter.items())[:20])


Количество уникальных attempt_key: 5040
Пример распределения по попыткам: [((11549, 0), 5), ((11573, 1), 5), ((11574, 0), 5), ((11792, 0), 5), ((11888, 0), 5), ((11891, 0), 5), ((12317, 0), 5), ((12388, 0), 5), ((12476, 0), 5), ((12491, 0), 5), ((12494, 0), 5), ((12614, 0), 5), ((13011, 0), 5), ((13256, 0), 5), ((13281, 0), 5), ((14167, 0), 5), ((14558, 0), 5), ((15581, 0), 5), ((16400, 0), 5), ((16408, 0), 5)]
